In [1]:
import pandas as pd

In [2]:
iium = pd.read_csv('iium.csv')
iium.shape

(20512, 1)

In [3]:
from bs4 import BeautifulSoup

In [4]:
from tqdm import tqdm

links = []
for i in tqdm(range(len(iium))):
    soup = BeautifulSoup(iium.iloc[i].values[0])
    links.append(soup.a['href'])

100%|██████████| 20512/20512 [00:20<00:00, 993.82it/s] 


In [5]:
# !pip3 install newspaper3k

In [6]:
from newspaper import Article

In [10]:
def get_text(url):
    try:
        article = Article(url, language='id')
        article.config.request_timeout = 120
        article.download()
        article.parse()
        article.title
        return {'title': article.title, 'text': article.text}
    except Exception as e:
        return {'error': str(e), 'url': url}

def loop(strings):
    results = []
    for i in tqdm(range(len(strings))):
        results.append(get_text(strings[i]))
    return results

In [11]:
import multiprocess

In [12]:
%%time

results = multiprocess.multiprocessing(links, loop, cores = 26)

100%|██████████| 788/788 [6:26:40<00:00, 29.44s/it]


CPU times: user 1min 6s, sys: 1min 3s, total: 2min 10s
Wall time: 6h 26min 40s


In [13]:
import json

with open('iium.json', 'w') as fopen:
    json.dump(results, fopen)

In [15]:
len([i for i in results if 'error' in i]) / len(results)

0.1294851794071763